# Hämta något på nätet (en webcrawler)
Vi ska börja med en simpel datainsamlare som plockar upp information från VGR:s externwebb. I slutänden kan man ställa frågor mot denna information när vi sparat ner resultatet. Lite som en sökmotor.

## 1. Anslut till vgregion.se och hämta länkar
Samlar in länkar från en webbadress. 

In [1]:
import requests
from bs4 import BeautifulSoup

min_startpunkt = 'https://www.vgregion.se'
lista_med_lankar = list()

r = requests.get(min_startpunkt)
soup = BeautifulSoup(r.text)

for a in soup.find_all('a', href=True):
    lista_med_lankar.append(a['href']) # sparar undan länkens adress
    
    print("Hittade länkar:", a['href'])
    

Hittade länkar: #main-content
Hittade länkar: /jobba-i-vgr/
Hittade länkar: /halsa-och-vard/
Hittade länkar: /regional-utveckling/
Hittade länkar: /kultur/
Hittade länkar: /kollektivtrafik/
Hittade länkar: /politik/
Hittade länkar: /aktuellt/
Hittade länkar: /kontakt-vgr/
Hittade länkar: /om-vgr/
Hittade länkar: /teckensprak/
Hittade länkar: /aktuellt/press/
Hittade länkar: /webbkarta/
Hittade länkar: /om-vgr/om-webbplatsen/tillganglighetsredogorelse/
Hittade länkar: https://www.vgregion.se/en/
Hittade länkar: https://www.vgregion.se/fi/
Hittade länkar: https://www.vgregion.se/rom/
Hittade länkar: https://www.vgregion.se/se/
Hittade länkar: https://www.vgregion.se/yi/
Hittade länkar: https://www.vgregion.se/fit/
Hittade länkar: /
Hittade länkar: https://www.vgregion.se/en/
Hittade länkar: https://www.vgregion.se/fi/
Hittade länkar: https://www.vgregion.se/rom/
Hittade länkar: https://www.vgregion.se/se/
Hittade länkar: https://www.vgregion.se/yi/
Hittade länkar: https://www.vgregion.se

## 2. Hämta sidtitel och text för de 20 första länkarna
Det är lite så här en sökmotor som Google indexerar webben och sedan låter oss söka i deras index efter saker vi letar efter. Vi hämtar innehållet på de första 20 sidorna och sparar ner resultatet.

In [2]:
# klipper ner listan så vi bara har de 20 första
kortare_lista = lista_med_lankar[:20]

mitt_resultat = dict() # här kommer vi spara ner våra träffar

"""
Kollar om http finns i länkens adress, om det inte gör det så läggs den till.
Sedan hämtar vi in information om alla dessa sidor.
"""
for sida in kortare_lista:
    if sida.startswith('http') == False:
        sida = f'https://www.vgregion.se{sida}'
    
    # hämtar in webbsidans källkod och skriver ut sidtiteln så vi ser att det funkar
    r = requests.get(sida)
    soup = BeautifulSoup(r.text)
    print(f'Hämtad URL: {sida}')
    print(f'Sidans titel: {soup.title.text}')
    print('###################################')
    
    # sparar undan sidans information
    # soup.text nedan sparar ned all läsbar text som syns på respektive webbsida
    mitt_resultat[sida] = [soup.title.text, soup.text]
    
print('Hämtningen är nu klar! Hoppa ner till nästa cell.')

Hämtad URL: https://www.vgregion.se#main-content
Sidans titel: Västra Götalandsregionen - Västra Götalandsregionen
###################################
Hämtad URL: https://www.vgregion.se/jobba-i-vgr/
Sidans titel: Jobba i VGR - Västra Götalandsregionen
###################################
Hämtad URL: https://www.vgregion.se/halsa-och-vard/
Sidans titel: Hälsa och vård - Västra Götalandsregionen
###################################
Hämtad URL: https://www.vgregion.se/regional-utveckling/
Sidans titel: Regional utveckling - Västra Götalandsregionen
###################################
Hämtad URL: https://www.vgregion.se/kultur/
Sidans titel: Kultur - Västra Götalandsregionen
###################################
Hämtad URL: https://www.vgregion.se/kollektivtrafik/
Sidans titel: Kollektivtrafik - Västra Götalandsregionen
###################################
Hämtad URL: https://www.vgregion.se/politik/
Sidans titel: Politik - Västra Götalandsregionen
###################################
Hämtad UR

## 3. Leta bland sidorna vi samlat in
Här kommer själva sökmomentet in. Vi börjar med ett ord vi letar efter och sedan en for-loop som går igenom resultatet för att leta om ordet finns i den data vi samlat in.

Vi har några if-satser som påverkar hur svaret ser ut. Vi gör skillnad på att ett ord kan verka vara en "fullträff" och därmed extra relevant i denna simpla tillämpning.

In [3]:
finns_ordet = 'kontakt' # ordet kan du byta ut till lite vad du vill för att kolla efter andra saker

for nyckel in mitt_resultat:
    traff = False
    
    if finns_ordet.lower() in mitt_resultat[nyckel][0].lower():
        print(f'Woho! Ordet "{finns_ordet}" finns i sidtiteln på adressen "{nyckel}"')
        traff = True
        
    if traff == True and finns_ordet.lower() in mitt_resultat[nyckel][1].lower():
        print(f'Ok! Ordet "{finns_ordet}" finns i brödtexten på adressen "{nyckel}"')
        print("SICKEN FULLTRÄFF! Ordet finns både i titel och brödtext!")
    elif finns_ordet.lower() in mitt_resultat[nyckel][1].lower():
        print(f'Ok! Ordet "{finns_ordet}" finns i brödtexten på adressen "{nyckel}"')
        traff = True
        
    if traff == False:
        print(f'Nope! Finns inte på adressen "{nyckel}"')
    
    print('###################################')

Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se#main-content"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/jobba-i-vgr/"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/halsa-och-vard/"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/regional-utveckling/"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/kultur/"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/kollektivtrafik/"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/politik/"
###################################
Ok! Ordet "kontakt" finns i brödtexten på adressen "https://www.vgregion.se/aktuellt/"
#########################